In [1]:
import pyodbc
import pandas as pd
import re
import geocoder

In [14]:
conn = pyodbc.connect('DSN=SQLAmlin;UID=MSAmlin;PWD=password')
cur = conn.cursor()
data = 'select * from initial.hampshire'

In [20]:
df = pd.read_sql(data, conn)

In [4]:
df

,IncidentID,Index,Post Date,Title,Incident Text,Address
0,1,None,01/01/2014 21:21,Fire,"Around 38 firefighters from Hightown, Botley, ...",\r\n\r\r\n\tBotley RoadWest EndHampshireSO30Un...
1,2,None,01/01/2014 21:21,Fire,"Around 38 firefighters from Hightown, Botley, ...","<div class=""addressDisplay"">\r\r\n\tBotley Roa..."
2,3,None,01/04/2014 21:55,Fire,Firefighters have this evening dealt with a sm...,\r\n\r\r\n\tCommercial RoadTottonHampshireSO40...
3,4,None,01/04/2014 21:55,Fire,Firefighters have this evening dealt with a sm...,"<div class=""addressDisplay"">\r\r\n\tCommercial..."
4,5,None,01/04/2016 12:07,Fire - Hayling Island,"Firefighters from Hayling Island, Havant, Emsw...",\r\n\r\r\n\tSouthwood RoadHayling IslandPO11Un...
5,6,None,01/04/2016 12:07,Fire - Hayling Island,"Firefighters from Hayling Island, Havant, Emsw...","<div class=""addressDisplay"">\r\r\n\tSouthwood ..."
6,7,None,01/04/2018 12:54,Crews tackle bin chute fire at Portsmouth high...,"Crews from Southsea, Cosham, Portchester, Fare...","\r\n\r\r\n\tBrownlow CloseBuckland, Portsmouth..."
7,8,None,01/04/2018 12:54,Crews tackle bin chute fire at Portsmouth high...,"Crews from Southsea, Cosham, Portchester, Fare...","<div class=""addressDisplay"">\r\r\n\tBrownlow C..."
8,9,None,01/06/2016 22:51,Fire Domestic - Havant,Two crews from Havant and one from Cosham atte...,\r\n\r\r\n\tSoberton RoadHavantHampshirePO9 2E...
9,10,None,01/06/2016 22:51,Fire Domestic - Havant,Two crews from Havant and one from Cosham atte...,"<div class=""addressDisplay"">\r\r\n\tSoberton R..."


In [5]:
df['part_addr'] = 'addr'

In [22]:
df.shape

(1716, 6)

In [24]:
df['part_addr'] = df['Address'].map(lambda x: x.replace('\n','').replace('\r','').replace('\t','').replace(' ', '') if x is not None else x)
df['part_addr'] = df['part_addr'].map(lambda x: re.sub(r"([A-Z])", r" \1", x).strip().replace(' United Kingdom', '') if x is not None else x)

In [25]:
for i in range(0, len(df)):
    df['part_addr'][i] = re.sub(r"([A-Z])", r" \1", df['part_addr'][i]).strip().replace(' United Kingdom', '')

C:\Users\Student\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


TypeError: expected string or bytes-like object

In [ ]:
df.head()

In [19]:
df.iloc[2]['Address']

'\r\n\r\r\n\tCommercial RoadTottonHampshireSO40United Kingdom\r\n'

## NEED TO RUN THE GEOCODE FUNCTION BELOW WHEN WE GET ACCESS

In [81]:
def get_full_address(part_addr):
    '''Takes an address and returns a dictionary containing the components of the address.
    Works best with a road name and town'''
    
    # This outer try/except block ensures that a dictionary of the 
    # correct form is returned if the geocoder bit doesn't work
    # so that key errors won't stop this from being applied to a dataframe column
    try:
        addr_dict = {}
        
        g = geocoder.osm(part_addr)
        try:
            addr_dict['HouseNumber'] = g.json['raw']['address']['house_number'] 
        except KeyError:
            addr_dict['HouseNumber'] = None
        
        try:
            addr_dict['Road'] = g.json['raw']['address']['road']
        except KeyError:
            addr_dict['Road'] = None
        
        # The 'town' bit of the address is sometimes stored under different keys in the json
        try:
            addr_dict['Town'] = g.json['raw']['address']['town']
        except KeyError:
            try:
                addr_dict['Town'] = g.json['raw']['address']['suburb']
            except KeyError:
                try:
                    addr_dict['Town'] = g.json['raw']['address']['suburb']
                except KeyError:
                    try:
                        addr_dict['Town'] = g.json['raw']['address']['city']
                    except:
                        addr_dict['Town'] = None
        
        try:
            addr_dict['Postcode'] = g.json['raw']['address']['postcode']
        except KeyError:
            addr_dict['Postcode'] = None
            
        try:
            addr_dict['Lat'] = g.json['lat']
        except KeyError:
            addr_dict['Lat'] = None
        
        try:
            addr_dict['Long'] = g.json['lng']
        except KeyError:
            addr_dict['Long'] = None
        
        return addr_dict    
        
    except:
        return {'HouseNumber': 'Location not found', 'Road': 'Location not found', 'Town': 'Location not found', 'Postcode': 'Location not found', 'Lat': 'Location not found', 'Long': 'Location not found'}